# Leveraging Intel Optimizations with Hugging Face for Enhanced Model Performance

<img src="https://www.developer-tech.com/wp-content/uploads/sites/3/2023/08/intel-pytorch-foundation-ai-development-artificial-intelligence-coding-programming-machine-learning.jpg" alt="Alt Text" style="width: 400px;"/>

This notebook serves as an introduction to utilizing IPEX for fine-tuning a pre-trained model, specifically focusing on the `distilbert-base-uncased` model for multi-class emotion classification in text. In part two of this lab you will learn how to contribute the open source Hugging Face model hub.  

## Why This is Important

Understanding how to leverage Intel optimizations is crucial for developers looking to maximize computational efficiency and performance. By integrating IPEX with Hugging Face's API, we can significantly enhance training speeds, especially when utilizing mixed precision training with FP32 and BF16. This notebook will demonstrate these capabilities practically, offering insights into:

- How to enable IPEX within Hugging Face's `TrainingArguments` and training functions.
- Comparing training speeds and efficiencies between IPEX-enabled and standard training processes.
- Performing inference to assess the model's accuracy in classifying emotions.

## Acquiring the Learnings

Through step-by-step instructions, hands-on examples, and comparative analyses, this workshop will equip you with the skills to effectively integrate Intel's optimizations into your NLP projects using Hugging Face. Let's dive into the world where cutting-edge language processing meets optimized computational performance.


#### Environment Setup and Dependencies Installation

This cell prepares our working environment. It sources Intel oneAPI for optimal performance on Intel hardware (optional based on your setup) and installs specific versions of essential libraries: `transformers`, `torch`, and `intel_extension_for_pytorch`. These installations ensure we have the necessary tools to leverage Intel's optimizations.

In [1]:
!source /opt/intel/oneapi/setvars.sh #comment out if not running on Intel Developer Cloud Jupyter
!pip install transformers==4.35.2
!pip install torch==2.1.0
!pip install intel_extension_for_pytorch==2.1.0
!pip install datasets==2.16.1
!pip install accelerate==0.26.0

/bin/bash: /opt/intel/oneapi/setvars.sh: No such file or directory
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 790.9/790.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 34.8 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement torch==2.1.0 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0)
ERROR: No matching distribution found for torch==2.1.0
ERROR: Could not find a version that satisfies the requirement intel_extension_for_pytorch==2.1.0 (from versions: none)
ERROR: No matching distribution found for intel_extension_for_pytorch==2.1.0
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 52.2 MB/s eta 0:00:00:00:0100:

#### Loading Libraries and Packages

In this cell, we import the core libraries that will be used throughout the notebook. This setup is crucial as it prepares our Python environment with all the necessary tools for our tasks.

- `from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments`: We import key components from the Hugging Face Transformers library. `AutoModelForSequenceClassification` and `AutoTokenizer` are used for loading the model and tokenizer, respectively. `Trainer` and `TrainingArguments` are essential for setting up and running our model training.
- `from datasets import load_dataset`: This import from the `datasets` library allows us to easily load and preprocess datasets available in Hugging Face's datasets hub.
- `import numpy as np`: Numpy is a fundamental package for scientific computing in Python. It provides support for arrays, mathematical operations, and various utility functions.
- `from sklearn.metrics import accuracy_score`: We import the `accuracy_score` function from Scikit-Learn to calculate the accuracy of our model predictions during evaluation. This metric will help us quantify the performance of our fine-tuned model.

Overall, this cell lays the foundation for our machine learning tasks by equipping us with the necessary libraries and modules.t.

In [ ]:
# Import necessary libraries
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score
import torch 

#### Dataset Loading

Here, we load the `emotion` dataset from Hugging Face's datasets library. This dataset will be used for training and evaluating our DistilBERT model, providing a practical context for emotion classification in text.k..


In [ ]:
# Load the dataset
dataset = load_dataset("emotion")

#### Model and Tokenizer Initialization

In this cell, we initialize the `distilbert-base-uncased` model and its corresponding tokenizer for sequence classification. This setup is the first step in preparing our model for fine-tuning on the emotion classification task..


In [ ]:
# Load a pre-trained BERT model and tokenizer
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)
tokenizer = AutoTokenizer.from_pretrained(model_name)

#### Data Preprocessing

Data preprocessing is essential for model training. We define and apply a preprocessing function that tokenizes our text data, making it compatible with the DistilBERT model's input requirements.
.


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Apply preprocessing
encoded_dataset = dataset.map(preprocess_function, batched=True)

#### Training with IPEX

This cell is where the integration of Intel Extension for PyTorch (IPEX) comes into play. We define training arguments, including enabling BF16 and IPEX, and set up our Hugging Face trainer. The model is then trained on the emotion dataset, utilizing the enhanced capabilities provided by IPEX..


In [ ]:
import intel_extension_for_pytorch as ipex

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    bf16=True, 
    use_ipex=True,
    no_cuda=True,
)

# Define the trainer
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=-1)
    return {'accuracy': accuracy_score(labels, preds)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

#### Model Evaluation

Post-training, we evaluate the model's performance on the validation dataset. This evaluation will give us insights into the effectiveness of our training and the accuracy of the model in emotion classification.

In [ ]:
trainer.evaluate()

#### Inference and Testing

Finally, we test the fine-tuned model's inference capabilities on new sentences. This step involves preprocessing the test sentences, performing predictions, and mapping these predictions to human-readable labels. It allows us to visually inspect the model's ability to classify emotions in various text inputs.

In [ ]:
# Define test sentences
test_sentences = [
    "I am feeling incredibly happy and joyful today!",
    "I am so sad and down.",
    "I have mixed feelings about this.",
    "This is absolutely terrifying!",
]

# Preprocess the test sentences
encoded_input = tokenizer(test_sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')

# Predict using the fine-tuned model
with torch.no_grad():
    predictions = model(**encoded_input)

# Convert predictions to human-readable labels
predicted_labels = np.argmax(predictions.logits.numpy(), axis=1)

# Mapping for the 'emotion' dataset labels
label_map = {0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"}

# Print predictions
for sentence, label_id in zip(test_sentences, predicted_labels):
    print(f"Sentence: '{sentence}' - Emotion Prediction: {label_map[label_id]}")


# Conclusion and Discussion

### Conclusion

Throughout this workshop, we have explored the integration of Intel optimizations with Hugging Face's powerful Transformers library. By fine-tuning the DistilBERT model with the support of Intel Extension for PyTorch, we observed enhanced training speeds and efficient utilization of computational resources, especially notable in mixed precision training scenarios.

### Discussion

The exercise showcased not only the technical prowess of combining Hugging Face with Intel optimizations but also highlighted the practical benefits such as reduced training times and resource efficiency. This understanding is pivotal for developers working on NLP tasks, seeking to optimize model performance on Intel hardware. As AI and NLP continue to evolve, harnessing these optimizations will be key in developing more efficient and powerful AI applications.